In [2]:
import pymysql
from sqlalchemy import create_engine

url = 'bigdata.cpnqnxkxfbaf.ap-northeast-2.rds.amazonaws.com' 
usr = 'bigdata'
pwd = 'bigdata2020'
db = 'bigdata'
charset = 'utf8'
pymysql.install_as_MySQLdb()
engine = create_engine(f'mysql://{usr}:{pwd}@{url}:3306/{db}')

### Rawdata 다운로드 및 저장

#### 원본데이터 만들기

In [3]:
import pandas as pd
import os

# 패스 만들기
path = 'E:/BD_Class/file'
try:
    if not(os.path.isdir(path)):
        os.makedirs(os.path.join(path))
except OSError as e:
    if e.errno != errno.EXIST:
        print("Failed to create directory!!!!!")
        raise


sql = 'select * from chome;'
df = pd.read_sql_query(sql,engine)
df.to_csv(f'{path}/유치원db_raw.csv',encoding='euc-kr')

In [34]:
df.columns

Index(['sido', 'sigungu', 'name', 'ty_pe', 'ops', 'post', 'adr', 'phone',
       'fax', 'rooms', 'sqm', 'playground', 'teachers', 'no_capa', 'no_enroll',
       'lng', 'lon', 'bus', 'web', 's_d', 'brk_s_d', 'brk_e_d', 'e_d',
       'ops_day', 'dong'],
      dtype='object')

In [28]:
df.sigungu.unique()

array(['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구'], dtype=object)

In [53]:
df.dong.value_counts()

명동        6657
우면동       4758
상계동       4243
장지동       3572
개포동       3498
          ... 
대현동          6
동작동          6
충무로5가        2
성북동          2
영등포동8가       1
Name: dong, Length: 628, dtype: int64

In [36]:
len(df.dong.unique()) #동 개수 확인

629

### 세부 다운로드 및 저장

In [87]:
#SQL문을 입력하고 끌어옴
sql =f'select ops_day,sigungu,dong,sum(no_capa),sum(no_enroll) from chome group by ops_day, dong;'
mdata = pd.read_sql_query(sql,engine)

### 동별 시계열데이터 저장

In [88]:
mdata

,ops_day,sigungu,dong,sum(no_capa),sum(no_enroll)
0,2016-09-30,용산구,가락1동,257.0,191.0
1,2016-09-30,종로구,가락2동,1170.0,1046.0
2,2016-09-30,광진구,가락동,1793.0,1516.0
3,2016-09-30,강서구,가락본동,317.0,289.0
4,2016-09-30,구로구,가리봉동,65.0,60.0
...,...,...,...,...,...
29241,2020-08-31,용산구,후암동,1231.0,1016.0
29242,2020-08-31,동대문구,휘경1동,91.0,65.0
29243,2020-08-31,동대문구,휘경2동,480.0,337.0
29244,2020-08-31,동대문구,휘경동,296.0,247.0


In [95]:
# 패스 만들기
path = 'E:/BD_Class/file'
try:
    if not(os.path.isdir(path)):
        os.makedirs(os.path.join(path))
except OSError as e:
    if e.errno != errno.EXIST:
        print("Failed to create directory!!!!!")
        raise

if not(os.path.isdir(f'{path}/dong')):
        os.makedirs(os.path.join(f'{path}/dong'))

for dong in mdata.dong.unique():
    log = mdata[mdata.dong==f'{dong}']   
    log.to_csv(f'{path}/dong/유치원db_{dong}.csv',encoding='euc-kr',index=False)

In [80]:
df

,sido,sigungu,name,ty_pe,ops,post,adr,phone,fax,rooms,...,lng,lon,bus,web,s_d,brk_s_d,brk_e_d,e_d,ops_day,dong
0,서울특별시,종로구,이화어린이집,민간,정상,3123,서울특별시 종로구 율곡로20길 3 (충신동),02-747-2686,2,6,...,37.489,127.038,운영,None,2004-05-03,None,None,None,2016-09-30,충신동
1,서울특별시,종로구,삼청어린이집,국공립,정상,3049,서울특별시 종로구 삼청로7길 26 (팔판동),02-723-0155,2,3,...,37.584,126.980,미운영,http://cafe.naver.com/samchung1004,1999-09-01,None,None,None,2016-09-30,삼청동
2,서울특별시,종로구,창신어린이집,사회복지법인,정상,3105,서울특별시 종로구 창신5길 22 (창신동),02-743-6302,2,4,...,37.574,127.012,운영,http://changsin.ukid.kr,1978-12-01,None,None,None,2016-09-30,창신1동
3,서울특별시,종로구,아동회관어린이집,국공립,정상,3094,서울특별시 종로구 지봉로13길 14 아동회관어린이집,02-763-6038,50,6,...,37.578,127.012,미운영,None,1988-10-01,None,None,None,2016-09-30,창신동
4,서울특별시,종로구,부암어린이집,국공립,정상,3019,서울특별시 종로구 세검정로6다길 10-7,02-396-6226,2,5,...,37.602,126.963,운영,www.buarm.co.kr,1998-07-29,None,None,None,2016-09-30,신영동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281353,서울특별시,강동구,구립아르테온보나어린이집,국공립,정상,5274,서울특별시 강동구 고덕로 360 고덕아르테온 339동 인근 관리동,02-442-2022,0,4,...,37.566,126.978,미운영,None,2020-02-20,None,None,None,2020-08-31,명동
281354,서울특별시,강동구,구립아르테온보예어린이집,국공립,정상,5274,서울특별시 강동구 고덕로 360 고덕아르테온아파트 319동 1층(상일동),02-6925-5642,2,6,...,37.555,127.168,미운영,None,2020-02-20,None,None,None,2020-08-31,명동
281355,서울특별시,강동구,이삭숲어린이집,민간,정상,5219,서울특별시 강동구 동남로82길 29-8 이삭숲어린이집,02-441-2914,2,6,...,37.561,127.159,운영,None,2020-05-06,None,None,None,2020-08-31,명동
281356,서울특별시,강동구,구립에코포레어린이집,국공립,정상,5300,서울특별시 강동구 천중로 253 이편한세상강동에코포레아파트 관리동 1층,02-470-0633,0,3,...,37.566,126.978,None,None,2020-06-01,None,None,None,2020-08-31,명동


In [94]:
# 패스 만들기
path = 'E:/BD_Class/file'
try:
    if not(os.path.isdir(path)):
        os.makedirs(os.path.join(path))
except OSError as e:
    if e.errno != errno.EXIST:
        print("Failed to create directory!!!!!")
        raise
if not(os.path.isdir(f'{path}/dong')):
        os.makedirs(os.path.join(f'{path}/dong'))
mdata.to_csv('E:/BD_Class/BD_Class/pd/dong/유치원db_capa.csv',encoding='euc-kr',index=False)